In [45]:
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import pandas as pd


In [46]:
# import pandas as pd

# def read_data(path):
#     df = pd.read_csv(path)
#     df = df.sample(frac=0.1, random_state=42)  # random_state for reproducibility
#     df = df.reset_index(drop=True)  # Optional: reset index after sampling
#     return df


In [47]:
# path='/home/abolfazl/Documents/CitizenJournal/citizen_journal/backend/fastapi_backend/HateSpeech/HateSpeechDatasetBalanced.csv'
# df=read_data(path)

### BertTokenizer: 
 Splits and converts your input text (like a sentence) into tokens that BERT can understand.

### BertModel:
 The actual pre-trained BERT model that turns the tokens into useful numeric vectors (called embeddings).

### bert-base-uncased:
 A version of BERT that has been trained on English text, where all words are lowercase.

### model.eval():
 Tells the model to run in inference mode (not training mode). This saves memory and speeds up processing.

In [48]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')
# model.eval()  # inference mode


In [49]:
# def get_cls_embedding(text):
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     cls_embedding = outputs.last_hidden_state[:, 0, :]  # [CLS] token
#     return cls_embedding.squeeze().numpy()  # shape: (768,)


In [50]:
# embeddings = []

# for text in tqdm(df['Content'], desc="Extracting BERT embeddings"):
#     try:
#         vector = get_cls_embedding(text)
#         embeddings.append(vector)
#     except Exception as e:
#         embeddings.append(None)  # in case of any failure


In [51]:
# # Convert the list of embeddings into a DataFrame
# embedding_df = pd.DataFrame(embeddings)

# # Rename the columns before merging
# embedding_df.columns = [f'embedding_{i}' for i in range(embedding_df.shape[1])]

# # Merge with the original DataFrame
# df = pd.concat([df, embedding_df], axis=1)


In [52]:
# output_path = '/home/abolfazl/Documents/CitizenJournal/citizen_journal/backend/fastapi_backend/HateSpeech/embedded_data.csv'
# df.to_csv(output_path, index=False)


In [53]:
import pandas as pd

# Load the saved dataset
df = pd.read_csv('/home/abolfazl/Documents/CitizenJournal/citizen_journal/backend/fastapi_backend/HateSpeech/embedded_data.csv')
df.head()

,Content,Label,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_758,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767
0,you should be deeply embarrassed... by not ful...,1,0.278104,0.333025,-0.294160,-0.145966,-0.485665,-0.786977,0.459131,0.544359,...,0.046933,-0.309165,-0.127641,-0.448980,-0.140215,0.193676,-0.034632,-0.019996,0.560498,0.340829
1,do not make me make you fall in love with a bi...,0,-0.133017,0.202193,-0.421173,-0.009662,-0.201444,-0.419096,0.644618,0.595702,...,0.270720,-0.357238,0.121133,-0.065833,0.152818,0.050402,0.077813,-0.305431,-0.050150,0.260495
2,trump america is anti immigrant sexual activit...,1,-0.138762,0.005806,-0.092694,-0.007501,-0.443966,-0.109705,0.614334,0.544135,...,-0.178341,-0.271666,0.061014,0.067001,0.656415,0.115183,-0.189214,-0.475422,0.309695,0.382447
3,you guys are clearly a pole smoker please get ...,1,0.406987,0.299873,0.009246,-0.191551,-0.434151,-0.539173,0.511990,0.659850,...,-0.038749,-0.406413,0.051500,-0.162931,0.342772,-0.140089,-0.001462,-0.489326,0.283278,0.042651
4,oh come along the only reason people like stri...,0,0.209989,0.294031,0.037456,-0.131755,-0.274220,-0.452337,0.039897,0.521467,...,-0.015279,-0.253148,-0.329311,-0.145785,0.218858,0.116727,-0.372996,-0.278894,0.474181,0.245281


In [54]:
# Features: columns embedding_0 to embedding_767
X = df[[f'embedding_{i}' for i in range(768)]].values

# Labels: assume your label column is 'label'
y = df['Label'].values


In [55]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [56]:
import tensorflow as tf
from keras import layers, models

from keras import regularizers

from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

model = tf.keras.Sequential([
    layers.Input(shape=(768,)),
    
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])



model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop]
)


Epoch 1/50
154/473 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6511 - loss: 0.6206

KeyboardInterrupt: 

In [ ]:
# Accuracy on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")


148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8028 - loss: 0.4205
Test Accuracy: 0.8028


In [ ]:
from sklearn.metrics import classification_report

y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

print(classification_report(y_test, y_pred))


148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           0       0.87      0.71      0.78      2347
           1       0.76      0.90      0.82      2375

    accuracy                           0.80      4722
   macro avg       0.81      0.80      0.80      4722
weighted avg       0.81      0.80      0.80      4722



In [ ]:
# # Save the model
# model.save('/home/abolfazl/Documents/CitizenJournal/citizen_journal/backend/fastapi_backend/HateSpeech/hate_speech_model.keras')


In [57]:
model_path='/home/abolfazl/Documents/CitizenJournal/citizen_journal/backend/fastapi_backend/HateSpeech/hate_speech_model.keras'

from keras.models import load_model
model=load_model(model_path)

In [68]:
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.95).astype(int).flatten()


148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [69]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.5493434985175772

Classification Report:
               precision    recall  f1-score   support

           0       0.52      1.00      0.69      2347
           1       0.98      0.11      0.19      2375

    accuracy                           0.55      4722
   macro avg       0.75      0.55      0.44      4722
weighted avg       0.75      0.55      0.44      4722

